In [20]:
from infernal import feature_extraction as fe
from infernal import datastructures as ds
import spacy

In [21]:
nlp = spacy.load('pt_core_news_lg')

In [22]:
#!python -m spacy download pt_core_news_lg
#!pip install zss rdflib

In [23]:
texto = '''Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.

"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.

O projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei. O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessórias, sem prejuízo à principal: suavizar as flutuações do nível de atividade econômica e fomentar o pleno emprego no País. O governo concordou com a redação da proposta, apesar de o BC ser historicamente contrário a ampliar o escopo da atuação.

Maia já reclamou outras vezes da falta de empenho e atuação do governo para se aprovar a medida. Na semana passada, acusou o presidente do Banco Central, Roberto Campos Neto, de ter vazado informações sobre conversa que os dois tiveram no dia da decisão do Comitê de Política Monetária (Copom), que manteve a taxa Selic em 2% ao ano.

Ao jornal O Estado de S. Paulo, Maia criticou a articulação do presidente do BC em alertar sobre os reflexos para a economia da dificuldade do Congresso em avançar com as votações da pauta de ajuste fiscal. Segundo o presidente da Câmara, Campos Neto tentou fazer uma articulação política, sem combinar, o que não seria papel dele, mas dos ministros da Economia, Paulo Guedes, e da articulação política, Luiz Eduardo Ramos.

Nesta sexta-feira, o presidente da Câmara lembrou que havia uma proposta de autonomia do BC semelhante na Câmara, mas que não foi votada, e disse que não comentou até agora sobre o tema porque não foi procurado pelo governo para falar sobre o assunto. "Se eu conseguisse conversar com alguém do governo, eu poderia te responder, mas ninguém me procura. Não vou conversar com a imprensa antes de conversar com o governo", disse.

Como o Broadcast (sistema de notícias em tempo real do Grupo Estado) mostrou na quinta-feira, deputados já se articulam na Câmara para modificar o projeto aprovado pelo Senado. O partido Novo, por exemplo, quer enxugar a proposta que recebeu aval dos senadores para reduzir os chamados acessórios que foram colocados para o Banco Central.

Sobre a reforma tributária, Maia deu sinais de que quer aprovar o projeto antes de deixar a presidência da Casa e acredita que com acordo pode fazer isso rapidamente.'''

texto

'Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ). Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.\n\n"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano. É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. "A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.\n\nO projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei. O texto mantém o controle dos preços como objetivo central, mas inclui ainda duas novas metas acessórias, sem prejuízo à principal: suav

In [24]:
textdoc = nlp(texto)

In [25]:
def paragraphs(document):
    start = 0
    for token in document:
        if token.is_space and token.text.count("\n") > 1:
            yield document[start:token.i]
            start = token.i
    yield document[start:]

In [26]:
from itertools import tee

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)    
    next(b, None)
    return zip(a, b)

In [27]:
list(enumerate(textdoc.sents))

[(0,
  Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ).),
 (1,
  Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.
  ),
 (2, "),
 (3,
  Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano.),
 (4,
  É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú.),
 (5, "),
 (6,
  A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou.
  ),
 (7,
  O projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei.),
 (8,
  O texto mantém o controle dos preços como objetivo central, 

In [28]:
sentencas = []

add_asspas = False
for s in textdoc.sents:
    if s.text == '"':
        add_asspas = True
    else:
        if add_asspas:
            sentencas.append('"'+s.text)
            add_asspas = False
        else:
            sentencas.append(s.text)
        

In [29]:
pares = [(s1,s2) for s1,s2 in pairwise(sentencas)]

pares

[('Defendida por agentes do mercado financeiro e uma das bandeiras da equipe econômica do governo Jair Bolsonaro, o projeto de autonomia do Banco Central (BC) deve avançar na Câmara só após a reforma tributária andar, no que depender do presidente da Casa, Rodrigo Maia (DEM-RJ).',
  'Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.\n\n'),
 ('Para ele, o projeto sobre a instituição presidida por Roberto Campos Neto não é urgente no curto prazo.\n\n',
  '"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano.'),
 ('"Aceito votar autonomia do Banco, aceito, é claro, votar os depósitos voluntários, mas aí temos que organizar melhor a pauta até o fim do ano.',
  'É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú.'),
 ('É só o governo ter boa vontade na reforma tributária", disse Maia, ao pa

In [30]:
extrator = fe.FeatureExtractor(both=True)

extrator.get_feature_names()

['Bleu T->H',
 'Bleu H->T',
 'Cosine TED',
 'Cosine TED / Length T',
 'Cosine TED / Length H',
 'Depedency Overlap T',
 'Depedency Overlap H',
 'Nominalization in H',
 'Nominalization in T',
 'Length Ratio',
 'Verb Structure T->H',
 'Verb Structure H->T',
 'Negated Verb',
 'Quantity Match',
 'Quantity Mismatch',
 'Sentence Cosine',
 'Simple TED',
 'TED / Length T',
 'TED / Length H',
 'Soft Overlap T->H',
 'Soft Overlap H->T',
 'Overlap T',
 'Overlap H',
 'Synonym Overlap T',
 'Synonym Overlap H',
 'Non-aligned Entity T',
 'Non-aligned Entity H',
 'Has Aligned Entity']

In [31]:
from infernal.datastructures import Entailment 
s1,s2 = pares[0]


par = ds.Pair(s1,s2, 0, Entailment.none)

par

In [32]:
s_ex = None
for i,s in enumerate(textdoc.sents):
    s_ex = s
    if i == 1:
        break
    print(dir(s))
    print(type(s))

dir(s_ex)

['_', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_fix_dep_copy', '_recalculate_indices', '_vector', '_vector_norm', 'as_doc', 'char_span', 'conjuncts', 'doc', 'end', 'end_char', 'ent_id', 'ent_id_', 'ents', 'get_extension', 'get_lca_matrix', 'has_extension', 'has_vector', 'kb_id', 'kb_id_', 'label', 'label_', 'lefts', 'lemma_', 'lower_', 'merge', 'n_lefts', 'n_rights', 'noun_chunks', 'orth_', 'remove_extension', 'rights', 'root', 'sent', 'sentiment', 'set_extension', 'similarity', 'start', 'start_char', 'string', 'subtree', 'tensor', 'text', 'text_with_ws', 'to_array', 'upper_', 'vector', 'vector_norm', 'vocab']
<class 'spacy.tokens.span.Span'>


['_',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_fix_dep_copy',
 '_recalculate_indices',
 '_vector',
 '_vector_norm',
 'as_doc',
 'char_span',
 'conjuncts',
 'doc',
 'end',
 'end_char',
 'ent_id',
 'ent_id_',
 'ents',
 'get_extension',
 'get_lca_matrix',
 'has_extension',
 'has_vector',
 'kb_id',
 'kb_id_',
 'label',
 'label_',
 'lefts',
 'lemma_',
 'lower_',
 'merge',
 'n_lefts',
 'n_rights',
 'noun_chunks',
 'orth_',
 'remove_extension',
 'rights',
 'root',
 'sent',
 'sentiment',
 'set_extension',
 'similarity',
 'start',
 'start_char',
 'string',
 'subtree',
 'tensor',
 'text',
 'text_with_ws',
 'to_array',
 'upper_',
 'vector',
 'vector_norm'

In [ ]:
from text_document import TextDocument

td = TextDocument(texto)

In [ ]:
for i, sent in enumerate(td.sentences):
    print()
    print("ID: ", str(i))   
    print(sent.tokens_dbpedia_metions)
    print("Texto Original: ", sent.text)    

In [20]:
import helper_tools as htools

#htools.get_dbpedia_mentions(texto)
dbpedia_mentions = []
dbpedia_mentions_entries = htools.get_dbpedia_entries(texto)
for key, entry in dbpedia_mentions_entries.items():
    if entry and key == 'raw_text':        
        dbpedia_mentions.append(entry['raw_text'].replace("'",''))

print(dbpedia_mentions)
print(dbpedia_mentions_entries)

AttributeError: 'str' object has no attribute 'keys'

In [ ]:
# Recuperar os tokens que constituem os spans relacionados
# as menções obtidas da dbpedia spotlight

ent1 = list(dbpedia_mentions_entries.values())[0]

tk0 = None
for t in td.sentences[0].annotated:
    if  t.idx == int(ent1['pos']):
        tk0 = t


sp1 = td.sentences[0].annotated[tk0.i:tk0.i+len(ent1['raw_text'].split())]
for t in sp1:
    print(type(t))
    print(t.pos_)
    print(t.text)

In [ ]:
from single_sentence import SingleSentence

for i,s in enumerate(td.sentences):
    if i == 1:
        break

id_sent = 1        
    
    
#for t in td.sentences[id_sent].list_fe:    
#    print(f"{t} -- {t.pos_} -- {t.dep_}")
    
#print(td.sentences[0].list_fe)    
#s_ano = td.nlp_processor(td.sentences[0].text)
#spacy.displacy.render(s_ano, style="dep", options = {"compact": True})

#print(td.sentences[1].list_fe)    
#s_ano = td.nlp_processor(td.sentences[1].text)
#spacy.displacy.render(s_ano, style="dep", options = {"compact": True})

In [ ]:
for i, sent in enumerate(td.sentences):
    print()
    print("ID: ", str(i))    
    print("Texto Original: ", sent.text)    
    print()    
    print("Lista de Foco Explícito: ", [s for s in sent.list_fe])    
    print()
    print("Lista de Entidades Nomeadas: ", sent.named_entities)    
    print()
    print("Lista de Menções DBPedia: ", str(sent.dbpedia_mentions))    
    #if i == 4:
    #    spacy.displacy.render(sent.anotated, style="dep", options = {"compact": True})    
    print("-"*100)
    
    

#s_disp = td.nlp_processor(td.sentences[4].text)
#spacy.displacy.render(s_disp, style="dep", options = {"compact": True})    

In [ ]:
#print(td.sentences[0].dbpedia_mentions[0])
for i, sent in enumerate(td.sentences):
    for text_token in sent.list_fe:    
        pos_token = sent.tokens_fe_pos_tags[text_token].pos_ if text_token in sent.tokens_fe_pos_tags.keys() else None
        print(text_token, pos_token)

In [ ]:
sent = td.sentences[0]

import itertools as itt

it_fe_pairs = itt.permutations(sent.list_fe, 2)


print([elem_fe_pair for elem_fe_pair in it_fe_pairs])

In [ ]:
import itertools as itt

it_fe_pairs = itt.permutations(sent.list_fe, 2)
head_elem = None
is_part_of_other = False
elems_to_exclude = set([])
for a,b in it_fe_pairs:    
    if len(a.split()) == 1:    
        if a in b:
            elems_to_exclude.add(a)
            is_part_of_other = True            
            
print(sent.list_fe,' - ', len(sent.list_fe))    
list_fe = list(set(sent.list_fe).difference(set(elems_to_exclude)))
print(list_fe,' - ', len(list_fe))    

In [ ]:
ss = td.sentences[3]

#print(dir(ss.anotated))
#ss.anotated.start

sentence_tokens = [token for token in ss.annotated]
type(sentence_tokens[0])
#sentence_tokens

In [ ]:
# Vinculate token and sentence
import spacy
from spacy.tokens import Token

def get_sentence(token):
    for sent in token.doc.sents:
        if sent.start <= token.i:
            return sent

# Add a computed property, which will be accessible as token._.sent
Token.set_extension('sent', getter=get_sentence)

nlp = spacy.load('en_core_web_sm')
doc = nlp(u'Sentence one. Sentence two.')
print(list(doc.sents))
print(doc[0]._.sent)
print(doc[-1]._.sent)

### Problema de segmentação de sentenças nas asspas que iniciam uma frase

Esse problema provavelmente, não dá pra resolver apenas alterando as marcações de início das sentenças, por que no caso de asspas que iniciam uma frase, o parser de dependências marca elas como ROOT e acho que isso faz com que a marcação fiquei incorreta, mas não se consiga alterar os marcadores de borda das sentenças. 

In [ ]:
def set_custom_boundaries(doc):        
    for token in doc[:-1]:        
        if token.text == "\"" and \
           token.is_sent_start and \
           token.is_sent_end:            
            doc[token.i+1].is_sent_start = False
    
    return doc

nlp.add_pipe(set_custom_boundaries, before="parser")


In [ ]:
def set_custom_boundaries(doc):        
    for token in doc[:-1]:        
        if token.text == "\"" and \
           token.is_sent_start and \
           token.is_sent_end:            
            doc[token.i+1].is_sent_start = False
    
    return doc

nlp.add_pipe(set_custom_boundaries, before="parser")


In [ ]:
nlp.remove_pipe('set_custom_boundaries')
#nlp.remove_pipe('sbd')

In [ ]:
from spacy.pipeline import SentenceSegmenter

for n,_ in nlp.pipeline:
    if 'sentencizer_hook' == n: 
        nlp.remove_pipe('sentencizer_hook')

def split_on_breaks(doc):
    start = 0
    seen_break = False
    for word in doc:
        if seen_break:
            yield doc[start:word.i-1]
            start = word.i
            seen_break = False
        elif word.text == '"' and \
             word.is_sent_start and \
             word.is_sent_end:
            seen_break = True
            #yield doc[start:word.i-1]
            #start = word.i+1
    if start < len(doc):
        yield doc[start:len(doc)]

sbd = SentenceSegmenter(nlp.vocab, strategy=split_on_breaks)
nlp.add_pipe(sbd, first=False)

In [ ]:
def get_sentences(text):
    doc = nlp(text)
    return (list(doc.sents)) # convert to string if required.

In [ ]:
text = '''"É só o governo ter boa vontade na reforma tributária", disse Maia, ao participar de evento organizado pelo Itaú. 
"A reforma tributária tem importância muito maior que autonomia do Banco Central", comentou. O projeto de autonomia do BC foi aprovado na terça-feira, 3, pelo Senado e agora precisa do aval dos deputados para virar lei.'''

#get_sentences(text)
doc = nlp(text)
sents = [sent.text for sent in doc.sents]
print("After:", sents)
print("tamanho:", len(sents))

In [ ]:
import split_utils as su
su.split_by_sentence(texto)

In [ ]:
for token in doc:
    #if doc[token.i].is_sent_start and \
    #   doc[token.i].is_sent_end and \
    #   doc[token.i].text == '"': 
    if doc[token.i].text == '"': 
        print(doc[token.i-1].is_sent_start, doc[token.i-1].is_sent_end, doc[token.i-1].text)                        
        print(doc[token.i].is_sent_start,   doc[token.i].is_sent_end, doc[token.i].text)            
        print(doc[token.i+1].is_sent_start, doc[token.i+1].is_sent_end, doc[token.i+1].text)
        print(doc[token.i].dep_)
        print()


In [ ]:
for token in doc[:-1]:            
        print(doc[token.i].is_sent_start,   doc[token.i].is_sent_end, doc[token.i].text)                    
        print(doc[token.i].pos_)
        print(doc[token.i].dep_)
        print()
